In [8]:
# group all 311 data by buildings, 
# get buildings' geoid using api and shapefiles,
# build a dictionary with key = (long, lat), and val= geoid
# process all 311 qualifying calls using the dictionary and retreive geoid

from shapely.geometry import Point, Polygon, MultiPolygon
import geopandas as gpd
import pandas as pd
shapefile_directory = r'D:\dataGasLeaks\NYC_Census_Tracts_for_2010_US_Census-shp\NYC_Census_Tracts_for_2010_US_Census.shp'
shapefile = gpd.read_file(shapefile_directory)

dff = pd.read_csv('D://dataGasLeaks//311_Service_Requests_updated.csv')
dff.shape
dff['count'] = 1
grouped_df = dff.groupby(['Created Date','Latitude', 'Longitude']).agg({'Borough': 'first', 'count': 'sum'})
grouped2_df = grouped_df.groupby(['Latitude', 'Longitude']).agg({'Borough': 'first', 'count': 'sum'})

## Dataframe with data from API: shorted block Fips to geoid, 
## Dataframe with data from shapefile: leftpad with zeros (6digits), find counties, create geoid

In [37]:
df1 = pd.read_csv('D://dataGasLeaks//311_Service_Requests_part13.csv')
df1 = df1[df1['ct'].notna()]
df1['ct'] = df1['ct'].astype(str).str[:11]

In [82]:
df2 = pd.read_csv('D://dataGasLeaks//311_Service_Requests_part9.csv')
df2 = df2[df2['ct'].notna()]

C:\Users\mskac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
shapefile_directory = r'D:\dataGasLeaks\table_2010_census_tracts_with_geoid\table_2010_census_tracts_with_geoid.shp'
shapefile = gpd.read_file(shapefile_directory)

In [123]:
df2.iloc[199999]

Unnamed: 0                200062
Latitude                 40.6089
Longitude               -74.0746
Borough            STATEN ISLAND
count                          9
ct                    3.6085e+14
boro               Staten Island
nta           Stapleton-Rosebank
Name: 200062, dtype: object

In [107]:
df2.to_csv(r'D:\dataGasLeaks\df2_api.csv')

In [126]:
for ind, row in df2[500000:len(df2)].iterrows():
    p = Point(row['Longitude'], row['Latitude'])
    for ind_shape, shape in shapefile.iterrows():
        if p.within(shape['geometry']):
            df2.at[ind, 'ct'] = str(shape['block_fips'])
            break

In [129]:
df = pd.concat([df1, df2[:200000], df2[500000:]])

In [130]:
df.to_csv(r'D:\dataGasLeaks\part1_ready.csv')

# Include 311 complaints into data

In [202]:
df = pd.read_csv(r'D:\dataGasLeaks\final_311_data.csv')
df_original = pd.read_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data.csv')

In [203]:
df = df.rename(columns={'year': 'incident_year', 'count': '311complaints'})
df = df.groupby(['geoid', 'incident_year']).agg({'311complaints': 'sum'})

df_new = pd.merge(df_original, df, on=['geoid','incident_year'], how='left')
df_new['population'] = df_new['gas_leaks']/df_new['gas_leaks_per_person']

In [215]:
df_new['311compl_per_person'] = df_new['311complaints']/df_new['population']
df_new.drop(['Unnamed: 0', '311complaints','population'], axis = 1, inplace = True)

In [217]:
df_new.to_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data1.csv')

## All years data

In [231]:
df = pd.read_csv(r'D:\dataGasLeaks\final_311_data.csv')
df_original = pd.read_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data.csv')
df_goal = pd.read_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data_all_years.csv')

In [233]:
df_original['population'] = df_original['gas_leaks']/df_original['gas_leaks_per_person']
df_original = df_original.groupby('geoid').agg({'population': 'sum'})

In [234]:
df = df.groupby('geoid').agg({'count': 'sum'})

df_new = pd.merge(df_original, df, on=['geoid'], how='left')
df_new['311compl_per_person'] = df_new['count']/df_new['population']

In [235]:
df_new = df_new.reset_index()
df_new.columns

Index(['geoid', 'population', 'count', '311compl_per_person'], dtype='object')

In [236]:
df_new.drop(['count','population'], axis = 1, inplace = True)
df_new.head()

,geoid,311compl_per_person
0,36005000200,0.261595
1,36005000400,0.191689
2,36005001600,0.145979
3,36005001900,0.644376
4,36005002000,0.141996


In [237]:
df_goal = pd.merge(df_goal, df_new, on=['geoid'], how='left')
df_goal

,Unnamed: 0,%housh_grandp_resp_for_grandch,avg_houshold_size,avg_bldg_age,gas_leaks,gas_leaks_per_person,geoid,lonely_housholder%,lonely_housholder_over65%,mean_houshold_income,median_age,nonfamily_housholds%,not_us_citizen%,total_housing_units,unemployed%,vacant_housing_units%,311compl_per_person
0,0,5.553494,3.730000,63.858957,38,0.001258,36005000200,17.439703,8.188002,75457.16667,38.300000,20.519481,13.337968,8880,4.217565,8.952703,0.261595
1,1,1.970487,3.031667,52.749939,49,0.001425,36005000400,16.311743,5.876116,85459.33333,38.016667,21.525139,7.444241,12399,3.986740,8.726510,0.191689
2,2,0.826587,2.815000,56.219298,99,0.002862,36005001600,27.158074,13.583298,47136.66667,37.466667,27.942054,16.517315,12290,3.173961,4.515867,0.145979
3,3,2.451161,2.425000,83.272222,38,0.002377,36005001900,37.670475,1.787689,47365.33333,31.333333,47.806856,15.832604,5763,8.438634,5.847649,0.644376
4,4,4.998347,2.981667,75.921555,314,0.005806,36005002000,29.499614,13.126860,34932.00000,33.416667,30.342775,9.354833,19106,5.571479,5.024600,0.141996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,2046,0.898835,2.738333,46.795396,46,0.001538,36085030301,28.313308,9.933046,77313.50000,35.233333,32.844171,8.059123,11350,3.501204,3.938326,0.158608
2047,2047,1.448905,3.231667,34.557914,38,0.001002,36085030302,13.219126,4.321145,85669.83333,35.483333,16.372624,13.835732,12509,2.778217,6.203533,0.178475
2048,2048,2.962029,2.600000,43.612317,97,0.005873,36085031901,24.452497,15.976052,35690.00000,30.050000,24.452497,9.676052,7213,3.645171,12.006100,0.189949
2049,2049,2.276439,2.886667,56.334072,22,0.000774,36085031902,21.355655,6.900776,54850.66667,28.733333,24.938751,9.278024,11354,2.469918,13.722036,0.280100


In [239]:
df_goal.to_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data_all_years1.csv')